In [8]:
import xml.etree.ElementTree as ET
import os
import re
import math
import numpy as np
import cv2
from tqdm import tqdm

path="./paipu2020.12.11-2021.12.1"


## 数据构成
1. 本家起手牌 4x34 14个通道为1
2. 舍牌信息 4家x24(最高舍牌数量)x34
3. dora指示牌 5x34(最多5个dora)
4. 所有可见牌 4x34(所有牌减去可见dora、可见鸣牌、自家手牌、舍牌)(暂时无法实现)
5. 场风、自风 2x34
6. 副露信息 4家x8(最多8副露)x4(1个副露4层)x34
7. 局顺 16x34
8. 本场 20x34（10进制100）
9. 供托 20x34（10进制100）
10. 分数 4x30x34（10进制1000）
11. 立直 4x34                   
12. 庄家信息 1x4x34


In [9]:
4+4*24+5+2+32*4+16+40+120+4+4

419

In [13]:
class paipuchuli():
    def __init__(self,xmlpath,path,count=0):
        image_path=os.path.join(path,"img")
        label_path=os.path.join(path,"label")
        self.count=count
        
        
        #读取xml
        tree = ET.parse(xmlpath)
        root = tree.getroot()
        paipu=[]
        flag=[]
        UNcount=0
        #确认你要分析的玩家位置 （suphx）
        for i in range(len(root)):
            #print(root[i])
            if root[i].tag=="UN" and UNcount==0:
                if root[i].attrib['n0']=="%E2%93%9D%53%75%70%68%78":
                    target=0
                elif root[i].attrib['n1']=="%E2%93%9D%53%75%70%68%78":
                    target=1
                elif root[i].attrib['n2']=="%E2%93%9D%53%75%70%68%78":
                    target=2
                elif root[i].attrib['n3']=="%E2%93%9D%53%75%70%68%78":
                    target=3
                else:
                    target=-1
                UNcount+=1
            #确认牌局(牌局指示标志 将一个半莊拆分为小局)
            if root[i].tag=='INIT':
                flag.append(i)
        
        #target 万一牌局没有要分析对象 引发错误
        assert(target!=-1),"targer error"
        #小局牌谱
        pat=re.compile(r"[TDUEVFWG]\d\d*")
        for i in range(len(flag)):
            if i!=len(flag)-1:
                paipu.append([])
                for j in range(flag[i+1]-flag[i]):
                    if re.match(pat, root[flag[i]+j].tag) != None:
                        paipu[i].append(root[flag[i]+j].tag)
                    else:
                        paipu[i].append(root[flag[i]+j])
            else:
                paipu.append([])
                for j in range(len(root)-flag[i]):
                    if re.match(pat, root[flag[i]+j].tag) != None:
                        paipu[i].append(root[flag[i]+j].tag)
                    else:
                        paipu[i].append(root[flag[i]+j])     
        
        self.target=target
        self.paipu =paipu
        labels=[]
        images=[]
        
        
        for i in range(len(paipu)):
            #print(i)
            label,image=self.generate(paipu[i])
            labels+=label
            images+=image
        assert(len(labels)==len(images)),"标签特征不匹配"
        #print(len(labels))
        for i in range(len(labels)):
            gray=images[i]*255
            cv2.imwrite(os.path.join(image_path,"{}.png".format(self.count)),gray)
            np.save(os.path.join(label_path,"{}.npy".format(self.count)),labels[i])
            self.count=self.count+1
        
        
        
        #print("end without error")
        #return output
    
    def generate(self,paipu_1):
        
        # 分数
        self.ten=[0]*4
        #标签
        self.label=[]
        
        #弃牌库
        self.qipaiku_0=[]
        self.qipaiku_1=[]
        self.qipaiku_2=[]
        self.qipaiku_3=[]
        
        #弃牌
        self.qipai=[]
    
        #dora
        self.dora=[]
        
        #reach
        self.reach=[0]*4
        
        #摸弃指示物
        mo_0=re.compile(r"[T]\d\d*")
        mo_1=re.compile(r"[U]\d\d*")
        mo_2=re.compile(r"[V]\d\d*")
        mo_3=re.compile(r"[W]\d\d*")
        qi_0=re.compile(r"[D]\d\d*")
        qi_1=re.compile(r"[E]\d\d*")
        qi_2=re.compile(r"[F]\d\d*")
        qi_3=re.compile(r"[G]\d\d*")
        
        #副露信息
        self.fulu_0=[-1]*8
        self.fulu_1=[-1]*8
        self.fulu_2=[-1]*8
        self.fulu_3=[-1]*8
        
        self.fulu_0_count=0
        self.fulu_1_count=0
        self.fulu_2_count=0
        self.fulu_3_count=0
        
        #输出图片
        images=[]
        
        
        for num,i in enumerate(paipu_1):
            #摸排弃牌处理
            if isinstance(i, str):
                if re.match(mo_0, i):
                    if self.target==0:
                        self.hai.append(i[1:])
                elif re.match(mo_1, i):
                    if self.target==1:
                        self.hai.append(i[1:])
                elif re.match(mo_2, i):
                    if self.target==2:
                        self.hai.append(i[1:])
                elif re.match(mo_3, i):
                    if self.target==3:
                        self.hai.append(i[1:])
                        
                elif re.match(qi_0, i) != None:
                    if self.target==0:
                        assert(len(self.hai)==14),"手牌长度错误"
                        image=self.image_process()
                            
                        
                        images.append(image)
                        self.label.append(i[1:])
                        self.hai.remove(i[1:])
                    self.qipaiku_0.append(i[1:])
                elif re.match(qi_1, i) != None:
                    if self.target==1:
                        assert(len(self.hai)==14),"手牌长度错误"
                        image=self.image_process()
                        
                        
                        images.append(image)
                        self.label.append(i[1:])
                        self.hai.remove(i[1:])
                    self.qipaiku_1.append(i[1:])
                elif re.match(qi_2, i) != None:
                    if self.target==2:
                        assert(len(self.hai)==14),"手牌长度错误"
                        image=self.image_process()
                        
                        
                        images.append(image)
                        self.label.append(i[1:])
                        self.hai.remove(i[1:])
                    self.qipaiku_2.append(i[1:])
                elif re.match(qi_3, i) != None:
                    if self.target==3:
                        assert(len(self.hai)==14),"手牌长度错误"
                        image=self.image_process()
                        
                        
                        images.append(image)
                        self.label.append(i[1:])
                        self.hai.remove(i[1:])
                    self.qipaiku_3.append(i[1:])
               
            else:
                #特殊关键字处理
                #1.牌局初始化信息 包括分数 庄家，场顺等等
                if i.tag=='INIT':
                    if self.target==0:
                        self.hai=list(i.attrib['hai0'].split(','))
                    elif self.target==1:
                        self.hai=list(i.attrib['hai1'].split(','))
                    elif self.target==2:
                        self.hai=list(i.attrib['hai2'].split(','))
                    elif self.target==3:
                        self.hai=list(i.attrib['hai3'].split(','))
                        
                    ten=i.attrib['ten'].split(',')
                    for t in range(4):
                        self.ten[t]=int(ten[t])
                    oya=i.attrib['oya']
                    self.oya=int(oya)
                    #oya-target:
                    zifeng=self.oya-self.target
                    if zifeng == 0:
                        #dong
                        self.zifeng=[27*4]
                    elif zifeng == -1 or 3:
                        #bei
                        self.zifeng=[30*4]
                    elif zifeng == -2 or 2:
                        #xi
                        self.zifeng=[29*4]
                    elif zifeng == -3 or 1:
                        #nan
                        self.zifeng=[28*4]
                    
                    
                    
                    ten=list(i.attrib['ten'].split(','))
                    for item in range(4):
                        self.ten[item]=int(ten[item])
                    
                    seed=list(i.attrib['seed'].split(','))
                    self.changshun=seed[0]
                    if int(self.changshun)<=3:
                        #dong
                        self.changfeng=[27*4]
                    elif 3<int(self.changshun)<=7:
                        #nan
                        self.changfeng=[28*4]
                    elif 7<int(self.changshun)<=11:
                        #xi
                        self.changfeng=[28*4]
                    elif 11<int(self.changshun)<=15:
                        #bei
                        self.changfeng=[28*4]
                        
                    self.benchang=seed[1]
                    self.gongtuo=seed[2]
                    self.dora.append(seed[5])
                #立直判断
                elif i.tag=="REACH":
                    if i.attrib['step']=='1':
                        self.reach[int(i.attrib['who'])]=1
                    elif i.attrib['step']=='2':
                        ten=i.attrib['ten'].split(',')
                        for item in range(4):
                            self.ten[item]=int(ten[item])
                #dora
                elif i.tag=="DORA":
                    self.dora.append(i.attrib['hai'])
                #副露信息
                elif i.tag=="N":
                    who=int(i.attrib['who'])
                    m=int(i.attrib['m'])
                    kui=(m&3)
                    if m&(1<<2):
                        #chi
                        kind=0
                        t=(m&0xFC00)>>10
                        r=t%3
                        t=math.floor(t/3)
                        t=math.floor(t/7)*9+(t%7)
                        t*=4
                        h=[t+4*0+((m&0x0018)>>3),t+4*1+((m&0x0060)>>5),t+4*2+((m&0x0180)>>7)]
                        if who == 0:
                            self.fulu_0[self.fulu_0_count]=h
                            self.fulu_0_count+=1
                        elif who == 1:
                            self.fulu_1[self.fulu_1_count]=h
                            self.fulu_1_count+=1
                        elif who == 2:
                            self.fulu_2[self.fulu_2_count]=h
                            self.fulu_2_count+=1
                        elif who == 3:
                            self.fulu_3[self.fulu_3_count]=h
                            self.fulu_3_count+=1
                        
                    elif m&(1<<3):
                        #peng
                        kind=1
                        t=(m&0xFE00)>>9
                        r=t%3
                        t=math.floor(t/3)
                        t*=4
                        h=[t,t,t]
                        if who == 0:
                            self.fulu_0[self.fulu_0_count]=h
                            self.fulu_0_count+=1
                        elif who == 1:
                            self.fulu_1[self.fulu_1_count]=h
                            self.fulu_1_count+=1
                        elif who == 2:
                            self.fulu_2[self.fulu_2_count]=h
                            self.fulu_2_count+=1
                        elif who == 3:
                            self.fulu_3[self.fulu_3_count]=h
                            self.fulu_3_count+=1
                            
                    elif m&(1<<4):
                        #jia gang
                        kind=2
                        added=(m&0x0060)>>5
                        t=(m&0xFE00)>>9
                        r=t%3
                        t=math.floor(t/3)
                        t*=4
                        h=[t]
                        if who == 0:
                            self.fulu_0[self.fulu_0_count]=h
                            self.fulu_0_count+=1
                        elif who == 1:
                            self.fulu_1[self.fulu_1_count]=h
                            self.fulu_1_count+=1
                        elif who == 2:
                            self.fulu_2[self.fulu_2_count]=h
                            self.fulu_2_count+=1
                        elif who == 3:
                            self.fulu_3[self.fulu_3_count]=h
                            self.fulu_3_count+=1
                    else:
                        #ming an gang
                        hai0=(m&0xFF00)>>8
                        if (not kui):
                            #ankang
                            kind=3
                            hai0=(hai0&~3)+3
                        else:
                            #mingkang
                            kind=4
                        t=math.floor(hai0/4)*4
                        h=[t,t,t,t]
                        if who == 0:
                            self.fulu_0[self.fulu_0_count]=h
                            self.fulu_0_count+=1
                        elif who == 1:
                            self.fulu_1[self.fulu_1_count]=h
                            self.fulu_1_count+=1
                        elif who == 2:
                            self.fulu_2[self.fulu_2_count]=h
                            self.fulu_2_count+=1
                        elif who == 3:
                            self.fulu_3[self.fulu_3_count]=h
                            self.fulu_3_count+=1
                            
                    if who ==self.target:
                        if kind==0 or kind==1:
                            if type(paipu_1[num-1]) == str:
                                self.hai.append(paipu_1[num-1][1:])
                            else:
                                self.hai.append(paipu_1[num-2][1:])
                            
                        elif kind==3 or kind==2:
                            self.hai.remove(str(h[0]))
            #print(len(self.hai))
            #assert(len(self.hai)==14),"手牌长度错误"
        #print(len(images))
        #print(len(self.label))
        assert(len(images)==len(self.label)),"图像标签不匹配"
        return self.label,images
    
    def paichuli1(self,hai):
        mhai=[ x//4 for x in hai]

        count=np.zeros((34))
        output=np.zeros((4,34))
        for i in mhai:
            count[i]+=1
            assert(1<=count[i]<5),"paiku out of range"
            output[int(count[i]-1)][i]=1
        return output
    
    def paichuli2(self,hai,aim_num):
        mhai=[ x//4 for x in hai]
        #layer0=np.zeros(34)
        for i,item in enumerate(mhai):
            if i==0:
                layer=np.zeros((1,34))
                layer[0][item ]=1
                total=layer
            else:
                layer=np.zeros(34)
                layer[item ]=1
                total=np.vstack((total,layer))
        if len(mhai)==0:
            total=np.zeros((aim_num,34))
            return total    
        
        
        if(total.shape[0]<aim_num):
            layer=np.zeros((aim_num-total.shape[0],34))
            total=np.vstack((total,layer))
        elif(total.shape[0]==aim_num):
            pass
        else:
            assert(1!=1),"chuli2 length error"

        return total        
      
    def paichuli_int(self,int_input,mode,num=-1):
        assert(mode==0 or mode==1),"mode error"
        if (mode==0):
            assert(num!=-1),"num error"
            if int_input<num:
                if int_input != 0:
                    output=np.vstack((np.ones((int_input,34)),np.zeros((num-int_input,34))))
                else:
                    output=np.zeros((num,34))
            return output
        elif (mode==1):
            assert(0<=int_input<1000),"out of range"
            if(int_input>=100):
                hundred=int_input//100
                output1=np.vstack((np.ones((hundred,34)),np.zeros((10-hundred,34))))
                ten=(int_input-hundred*100)//10
                output2=np.vstack((np.ones((ten,34)),np.zeros((10-ten,34))))
                single_digit=int_input-hundred*100-ten*10
                output3=np.vstack((np.ones((single_digit,34)),np.zeros((10-single_digit,34))))
                output=np.vstack((output1,output2,output3))
            elif (10<=int_input<100):
                hundred=0
                output1=np.zeros((10,34))
                ten=(int_input-hundred*100)//10
                output2=np.vstack((np.ones((ten,34)),np.zeros((10-ten,34))))
                single_digit=int_input-hundred*100-ten*10
                output3=np.vstack((np.ones((single_digit,34)),np.zeros((10-single_digit,34))))
                output=np.vstack((output1,output2,output3))
            elif (0<=int_input<10):
                hundred=0
                output1=np.zeros((10,34))
                ten=0
                output2=np.zeros((10,34))
                single_digit=int_input-hundred*100-ten*10
                output3=np.vstack((np.ones((single_digit,34)),np.zeros((10-single_digit,34))))
                output=np.vstack((output1,output2,output3))

        return output
    
    def image_process(self):
        #手牌处理
        result=[]
        #print(self.hai)
        if self.target==0:
            hai_int=[ int(x) for x in self.hai]
            kong_result=self.paichuli_int(0,0,4)
            shoupai_result= self.paichuli1(hai_int)
            result.append(shoupai_result)
            result.append(kong_result)
            result.append(kong_result)
            result.append(kong_result)
        elif self.target==1:
            hai_int=[ int(x) for x in self.hai]
            kong_result=self.paichuli_int(0,0,4)
            shoupai_result= self.paichuli1(hai_int)
            result.append(kong_result)
            result.append(shoupai_result)
            result.append(kong_result)
            result.append(kong_result)
        elif self.target==2:
            hai_int=[ int(x) for x in self.hai]
            kong_result=self.paichuli_int(0,0,4)
            shoupai_result= self.paichuli1(hai_int)
            result.append(kong_result)
            result.append(kong_result)
            result.append(shoupai_result)
            result.append(kong_result)
        elif self.target==3:
            hai_int=[ int(x) for x in self.hai]
            kong_result=self.paichuli_int(0,0,4)
            shoupai_result= self.paichuli1(hai_int)
            result.append(kong_result)
            result.append(kong_result)
            result.append(kong_result)
            result.append(shoupai_result)
        #弃牌处理
        
        qipaiku_0_int=[ int(x) for x in self.qipaiku_0]
        qipai_0_result= self.paichuli2(qipaiku_0_int,24)
        result.append(qipai_0_result)
        
        qipaiku_1_int=[ int(x) for x in self.qipaiku_1]
        qipai_1_result= self.paichuli2(qipaiku_1_int,24)
        result.append(qipai_1_result)
        
        qipaiku_2_int=[ int(x) for x in self.qipaiku_2]
        qipai_2_result= self.paichuli2(qipaiku_2_int,24)
        result.append(qipai_2_result)
        
        qipaiku_3_int=[ int(x) for x in self.qipaiku_3]
        qipai_3_result= self.paichuli2(qipaiku_3_int,24)
        result.append(qipai_3_result)
        
        #dora处理
        dora_int=[ int(x) for x in self.dora]
        dora_result=self.paichuli2(dora_int,5)
        result.append(dora_result)
        
        #自风 场风
        zifeng_int=[ int(x) for x in self.zifeng]
        zifeng_result=self.paichuli2(zifeng_int,1)
        result.append(zifeng_result)
        
        changfeng_int=[ int(x) for x in self.changfeng]
        changfeng_result=self.paichuli2(changfeng_int,1)
        result.append(changfeng_result)
        
        #副露信息
        fulu_0_result=[0]*8
        fulu_1_result=[0]*8
        fulu_2_result=[0]*8
        fulu_3_result=[0]*8
        assert(len(self.fulu_0)==8),"fulu error"
        assert(len(self.fulu_1)==8),"fulu error"
        assert(len(self.fulu_2)==8),"fulu error"
        assert(len(self.fulu_3)==8),"fulu error"
        for i,item in enumerate(self.fulu_0):
            if type(item)==list:
                fulu_0_result[i]=self.paichuli1(item)
            else:
                fulu_0_result[i]=np.zeros((4,34))
            result.append(fulu_0_result[i])
        for i,item in enumerate(self.fulu_1):
            if type(item)==list:
                fulu_1_result[i]=self.paichuli1(item)
            else:
                fulu_1_result[i]=np.zeros((4,34))
            result.append(fulu_1_result[i])
        for i,item in enumerate(self.fulu_2):
            if type(item)==list:
                fulu_2_result[i]=self.paichuli1(item)
            else:
                fulu_2_result[i]=np.zeros((4,34))
            result.append(fulu_2_result[i])
        for i,item in enumerate(self.fulu_3):
            if type(item)==list:
                fulu_3_result[i]=self.paichuli1(item)
            else:
                fulu_3_result[i]=np.zeros((4,34))
            result.append(fulu_3_result[i])
        #局顺
        jushun_result=self.paichuli_int(int(self.changshun),0,16)
        result.append(jushun_result)
        #本场
        benchang_result=self.paichuli_int(int(self.benchang),0,20)
        result.append(benchang_result)
        #供托
        gongtuo_result=self.paichuli_int(int(self.gongtuo),0,20)
        result.append(gongtuo_result)
        
        #分数
        ten_result_0=self.paichuli_int(int(self.ten[0]),1)
        result.append(ten_result_0)
        ten_result_1=self.paichuli_int(int(self.ten[1]),1)
        result.append(ten_result_1)
        ten_result_2=self.paichuli_int(int(self.ten[2]),1)
        result.append(ten_result_2)
        ten_result_3=self.paichuli_int(int(self.ten[3]),1)
        result.append(ten_result_3)

        #立直
        if self.reach[0]==1:
            reach_result_0=np.ones((1,34))
        else:
            reach_result_0=np.zeros((1,34))    
        if self.reach[1]==1:
            reach_result_1=np.ones((1,34))
        else:
            reach_result_1=np.zeros((1,34))
        if self.reach[2]==1:
            reach_result_2=np.ones((1,34))
        else:
            reach_result_2=np.zeros((1,34))
        if self.reach[3]==1:
            reach_result_3=np.ones((1,34))
        else:
            reach_result_3=np.zeros((1,34))
        result.append(reach_result_0)
        result.append(reach_result_1)
        result.append(reach_result_2)
        result.append(reach_result_3)
        
        #庄家
        oya_resylt=self.paichuli_int(int(self.oya),0,4)
        result.append(oya_resylt)
        return np.vstack(result)
    def my_count(self):
        return self.count

In [16]:
count=0
for i in tqdm(range(0,10242)):
    count=paipuchuli(os.path.join(path,str(i).rjust(5,'0')+".xml"),r"C:\Users\Paradox\Desktop\maj\train", count).my_count()
    #print(count)

100%|████████████████████████████████████████████████████████████████████████████| 10242/10242 [40:47<00:00,  4.18it/s]


In [18]:
files = os.listdir(r"C:\Users\Paradox\Desktop\maj\paipu2019.5.1-7.1")

In [20]:
for i in tqdm(range(len(files))):
    count=paipuchuli(os.path.join(r'C:\Users\Paradox\Desktop\maj\paipu2019.5.1-7.1',str(i).rjust(5,'0')+".xml"),r"C:\Users\Paradox\Desktop\maj\train", count).my_count()
    #print(count)

100%|██████████████████████████████████████████████████████████████████████████████| 2887/2887 [08:47<00:00,  5.48it/s]


In [ ]:
files = os.listdir(r"C:\Users\Paradox\Desktop\maj\paipu2019.3.5-5.1")
for i in tqdm(range(len(files))):
    if i <2500:
        pass
        #count=paipuchuli(os.path.join(r'C:\Users\Paradox\Desktop\maj\paipu2019.3.5-5.1',str(i).rjust(5,'0')+".xml"),r"C:\Users\Paradox\Desktop\maj\train", count).my_count()
    else:
        count=paipuchuli(os.path.join(r'C:\Users\Paradox\Desktop\maj\paipu2019.3.5-5.1',str(i).rjust(5,'0')+".xml"),r"C:\Users\Paradox\Desktop\maj\val", count).my_count()
    #print(count)

 90%|██████████████████████████████████████████████████████████████████████▏       | 2620/2913 [00:19<00:03, 93.93it/s]

In [95]:
#0-8 1-9m 9-17 1p-9p 18-26 1s-9s 27-30 d,n,x,b 31-33 b,f,z


[5, 9, 10, 15, 17, 17, 20, 21, 24, 24, 25, 31, 32]